In [1]:
import pandas as pd
import numpy as np

In [2]:
def p2f(x):
    return (float(x.strip('%'))/100 for col in columns)

In [15]:
metadata = pd.read_csv('data/metadata.csv', usecols = ['sample-id', 'group'])
nim_aminoacids = pd.read_csv('data/nim-aminoacids_400.csv', index_col=0)
#Per data description, limit the choice of amino acids as nutrients by the first 6
nim_aminoacids = nim_aminoacids.drop(nim_aminoacids.iloc[:, 7:], axis = 1)
nim_aminoacidsD = pd.read_csv('data/nim-aminoacidsD_400.csv', index_col=0)
nim_sugars = pd.read_csv('data/nim-sugars_400.csv', index_col=0)
nim_vitamins = pd.read_csv('data/nim-vitamins_400.csv', index_col=0)
taxonomy = pd.read_csv('data/taxonomy_400.csv', index_col=0)
taxonomy = taxonomy.replace('%', '', regex = True).astype(np.float64)

In [16]:
#merging all NIMs into one
nim = nim_aminoacids.merge(nim_aminoacidsD, how = 'inner', on = 'taxonomy')
nim = nim.merge(nim_sugars, how = 'inner', on = 'taxonomy')
nim = nim.merge(nim_vitamins, how = 'inner', on = 'taxonomy')

In [17]:
#Separating out normal & deviant sample-ids
normal = metadata.loc[metadata.group=='NORMAL']
normal = normal.drop(['group'], axis = 1)
normal_ids = normal.to_numpy()
#Flatten (un)necessary(?)
#normal_id = normal_id.flatten()
deviant = metadata.loc[metadata.group=='DEVIANT']
deviant = deviant.drop(['group'], axis = 1)
deviant_ids = deviant.to_numpy()
#Flatten (un)necessary(?)
#deviant_id = deviant_id.flatten()

In [26]:
#usage for ids array created above. 
normal_taxonomy = taxonomy[normal_ids.reshape(-1)]
deviant_taxonomy = taxonomy[deviant_ids.reshape(-1)]
normal_taxonomy = normal_taxonomy.to_numpy()
deviant_taxonomy = deviant_taxonomy.to_numpy()

In [27]:
#compute percentile for specific microbe
vlow_test = np.percentile(normal_taxonomy[0],10)
vhigh_test = np.percentile(normal_taxonomy[0],90)
print(vlow_test, vhigh_test)

5.4 18.3


In [41]:
#store vhigh/ vlow in arrays
vlow = np.percentile(normal_taxonomy, 10, axis=1) - 5
vhigh = np.percentile(normal_taxonomy, 90, axis=1) + 5
print(vlow[0], vhigh[0])

0.40000000000000036 23.3


In [44]:
def violations(u):
    if len(u.shape) == 1:
        u = u[np.newaxis, :]
    assert u.shape[1] == len(vlow) == len(vhigh)
    u = u / u.sum(axis=1, keepdims=True) * 100
    vio_low = u < vlow[np.newaxis, :]
    vio_high = u > vhigh[np.newaxis, :]
    vio = (vio_low | vio_high).astype(np.int32).sum(axis=1)
    return vio
print(violations(normal_taxonomy.transpose((1, 0))))
print(violations(deviant_taxonomy.transpose((1, 0))))

[0 1 0 ... 3 0 1]
[ 5  8  2  2  5  1  5  3  4  6  3  5  4  4  2  4  4  4  6  3  2  4  4  3
  7  2  2  5  3  3  7  3  4  2  6  1  3  2  6  4  2  4  3  2  3  6  5  4
  3  4  4  5  6  5 11  6  7  8  6  7  5  7  6  8  8  7  5  8  8  6  7  5
  8  6  5  8  7  6  8  8  7  5 10  6  5  8  7  8  9  6  2  3  2  3  2  2
  3  5  3  1  1  7  1  1  2  3  3  4  1  1  2  4  3  4  1  7  3  4  5  6
  3  1  2  8  2  1  3  3  1  2  4  4  4  2  5  1  4  3  1  1  7  5]


In [38]:
print(normal_taxonomy[:, 0])
print(deviant_taxonomy[:, 0])

[13.3 12.6  0.   6.7  3.   0.   0.9  0.1  0.   0.8  1.   0.5  0.9  0.6
  0.8  0.   5.3  0.7  0.   0.5  1.   0.   0.4  0.   0.6  0.   0.   0.5
  0.6  0.   0.5  3.5  0.   3.3  1.7  0.   0.   0.4  0.8  0.3  0.   0.
  0.   0.1  0.   3.6  0.   0.5  0.   0.4  0.   0.   3.9  0.   2.6  0.
  0.4  0.1  0.   0.   0.   0.   0.   0.   0.2  0.   0.1  1.1  0.   0.6
  0.5  0.   0.1  0.   0.2  0.   0.9  0.2  0.   0.1  0.   1.2  0.   0.
  0.4  0.6  0.   0.   0.4  0.   0.   0.   0.   0.   1.6  0.1  0.3  0.1
  0.   0.   0.   0.   0.5  0.1  0.   0.   0.   0.   0.8  0.   0.   0.
  1.6  0.1  0.   0.2  0.   0.1  0.2  0.   0.   0.   0.   0.1  0.   0.
  0.1  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   4.2  0.   0.7  0.   0.1  0.1
  0.   0.   0.   0.   0.4  0.   0.   0.3  0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.4  0.   0.2  0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.3  0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0. 